# Cifar100

## FedAvg, FedSGD, FedEla, FedProx, FedScaffold

Run following commands in the root path of `benchmark-lightly`.

### FedAvg

```bash
# Delete cache file
rm -rf /tmp/fedavg_cifar100.share
rm -rf /tmp/cifar100/fedavg_cifar100
rm -rf ./logs/cifar100/fedavg_cifar100
# Run
python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp benchmark/run.py\
    --fed_init_method file:///tmp/fedavg_cifar100.share\
    --task cifar100\
    --data_root benchmark/datasets/cifar100/data\
    --epochs 1\
    --rounds 20\
    --act_clts 100\
    --tst_act_clts 100\
    --max_acg_step -1\
    --optim fedavg\
    --follower_lr 1e-2\
    --leader_lr 10.0\
    --bz 10\
    --gpu\
    --log_level SUCCESS\
    --log_dir logs\
    --exp_name fedavg_cifar100\
    --seed 0
```

### FedSgd

```bash
# Delete cache file
rm -rf /tmp/fedsgd_cifar100.share
rm -rf /tmp/cifar100/fedsgd_cifar100
rm -rf ./logs/cifar100/fedsgd_cifar100
# Run
python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp benchmark/run.py\
    --fed_init_method file:///tmp/fedsgd_cifar100.share\
    --task cifar100\
    --data_root benchmark/datasets/cifar100/data\
    --epochs 1\
    --rounds 20\
    --act_clts 100\
    --tst_act_clts 100\
    --max_acg_step -1\
    --optim fedsgd\
    --follower_lr 1e-2\
    --leader_lr 10.0\
    --bz 10\
    --gpu\
    --log_level SUCCESS\
    --log_dir logs\
    --exp_name fedsgd_cifar100\
    --seed 0
```

### FedEla

```bash
# Delete cache file
rm -rf /tmp/fedela_cifar100.share
rm -rf /tmp/cifar100/fedela_cifar100
rm -rf ./logs/cifar100/fedela_cifar100
# Run
python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp benchmark/run.py\
    --fed_init_method file:///tmp/fedela_cifar100.share\
    --task cifar100\
    --data_root benchmark/datasets/cifar100/data\
    --epochs 1\
    --rounds 20\
    --act_clts 100\
    --tst_act_clts 100\
    --max_acg_step -1\
    --optim fedela\
    --follower_lr 1e-2\
    --leader_lr 10.0\
    --bz 10\
    --gpu\
    --log_level SUCCESS\
    --log_dir logs\
    --exp_name fedela_cifar100\
    --seed 0
```

### FedProx

```bash
# Delete cache file
rm -rf /tmp/fedprox_cifar100.share
rm -rf /tmp/cifar100/fedprox_cifar100
rm -rf ./logs/cifar100/fedprox_cifar100
# Run
python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp benchmark/run.py\
    --fed_init_method file:///tmp/fedprox_cifar100.share\
    --task cifar100\
    --data_root benchmark/datasets/cifar100/data\
    --epochs 1\
    --rounds 20\
    --act_clts 100\
    --tst_act_clts 100\
    --max_acg_step -1\
    --optim fedprox\
    --follower_lr 1e-2\
    --leader_lr 10.0\
    --bz 10\
    --gpu\
    --log_level SUCCESS\
    --log_dir logs\
    --exp_name fedprox_cifar100\
    --seed 0
```

### FedScaffold

```bash
# Delete cache file
rm -rf /tmp/fedscaffold_cifar100.share
rm -rf /tmp/cifar100/fedscaffold_cifar100
rm -rf ./logs/cifar100/fedscaffold_cifar100
# Run
python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp benchmark/run.py\
    --fed_init_method file:///tmp/fedscaffold_cifar100.share\
    --task cifar100\
    --data_root benchmark/datasets/cifar100/data\
    --epochs 1\
    --rounds 20\
    --act_clts 100\
    --tst_act_clts 100\
    --max_acg_step -1\
    --optim fedscaffold\
    --follower_lr 1e-2\
    --leader_lr 10.0\
    --bz 10\
    --gpu\
    --log_level SUCCESS\
    --log_dir logs\
    --exp_name fedscaffold_cifar100\
    --seed 0
```

## Plot Curves

In [ ]:
%matplotlib inline

from benchmark.utils.plot import plot

### Train Accuracy

In [ ]:
plot(
    files=[
        '../logs/cifar100/fedavg_cifar100/cifar100.json',
        '../logs/cifar100/fedsgd_cifar100/cifar100.json',
        '../logs/cifar100/fedela_cifar100/cifar100.json',
        '../logs/cifar100/fedprox_cifar100/cifar100.json',
        '../logs/cifar100/fedscaffold_cifar100/cifar100.json',
    ],
    labels=[
        'FedAvg',
        'FedSgd',
        'FedEla',
        'FedProx',
        'FedScaffold',
    ],
    attributes="accuracy",
    mode='train'
)

### Train Loss

In [ ]:
plot(
    files=[
        '../logs/cifar100/fedavg_cifar100/cifar100.json',
        '../logs/cifar100/fedsgd_cifar100/cifar100.json',
        '../logs/cifar100/fedela_cifar100/cifar100.json',
        '../logs/cifar100/fedprox_cifar100/cifar100.json',
        '../logs/cifar100/fedscaffold_cifar100/cifar100.json',
    ],
    labels=[
        'FedAvg',
        'FedSgd',
        'FedEla',
        'FedProx',
        'FedScaffold',
    ],
    attributes="loss",
    mode='train'
)

### Test Accuracy

In [ ]:
plot(
    files=[
        '../logs/cifar100/fedavg_cifar100/cifar100.json',
        '../logs/cifar100/fedsgd_cifar100/cifar100.json',
        '../logs/cifar100/fedela_cifar100/cifar100.json',
        '../logs/cifar100/fedprox_cifar100/cifar100.json',
        '../logs/cifar100/fedscaffold_cifar100/cifar100.json',
    ],
    labels=[
        'FedAvg',
        'FedSgd',
        'FedEla',
        'FedProx',
        'FedScaffold',
    ],
    attributes="accuracy",
    mode='test'
)

### Test Loss

In [ ]:
plot(
    files=[
        '../logs/cifar100/fedavg_cifar100/cifar100.json',
        '../logs/cifar100/fedsgd_cifar100/cifar100.json',
        '../logs/cifar100/fedela_cifar100/cifar100.json',
        '../logs/cifar100/fedprox_cifar100/cifar100.json',
        '../logs/cifar100/fedscaffold_cifar100/cifar100.json',
    ],
    labels=[
        'FedAvg',
        'FedSgd',
        'FedEla',
        'FedProx',
        'FedScaffold',
    ],
    attributes="loss",
    mode='test'
)